This workbook was run via a Julia v0.3.+ kernel and requires the installation of PyJulia.
Download it from github and build it by running _python setup.py install_.

Anaconda Python is recommended and the Julia kernel will need to have added the PyCall module
The only other module to add is Cosmology; this is only needed in order to run the example later in the workbook.

All the code snippets/sources are provided in the _code_ subdirectory.


# Calling Julia from Python

Import the Python module that we will need later

In [ ]:
import time as tm
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt

---

## Type inference examples ( _from earlier_ )

In [1]:
type(2 ** 3)

int

In [ ]:
type(2 ** -3)

In [ ]:
type(np.sqrt(1))

In [ ]:
type(np.sqrt(-1))

In [ ]:
type(np.sqrt(-1 + 0j))

In [ ]:
def fac(n):
 if n < 2:
   return 1
 else:
   return n*fac(n-1)

In [ ]:
fac(20)

In [ ]:
fac(21)

### Bring in the Julia interpretor

In [ ]:
from julia import Julia

In [ ]:
jl = Julia()

In [ ]:
jl.bessely0(1.5) * np.sin(1.5)

In [ ]:
x = [0.1*i for i in range(300)]

In [ ]:
y = [jl.gamma(0.015*(i+1)) * np.sin(0.15*i)  for i in range(300)]

In [ ]:
plt.plot(x,y)
plt.show()

---

### Series expansion for log(x), |x| < 1.0, is very slow as x -> 1.0

log(1+x) = x - x<sup>2</sup>/2 + x<sup>3</sup>/3 - x<sup>4</sup>/4 + x<sup>5</sup>/5 - . . .


In [ ]:
# Run it in Python

def slogp(x,n):
   if (n > 0 and abs(x) < 1):
      s = 0.0
      for i in range(n):
         j = i + 1
         s += ((-1)**i) * (x**j / float(j))
      return s
   else:
      raise ValueError('Illegal parameter values')

In [ ]:
slogp(0.99995,1000)

In [ ]:
t0 = tm.time(); slogp(0.99995,10000000); print (tm.time() - t0)

In [ ]:
# And now in Julia

slogj = jl.eval("""
function slog(x::Real,n::Integer)
  @assert abs(n) > 0
  @assert abs(x) < 1.0
  s = 0.0
  for i in 1:n
    s += (-1)^(i+1) * (x^i / i)
  end
  return s
end
""")

In [ ]:
slogj(0.99995,1000)

In [ ]:
slogj(0.99995,10000000)

In [ ]:
t0 = tm.time(); slogj(0.99995,10000000); print (tm.time() - t0)

---

### Vander function is difficult to compute as a vectorised process
<p>
A Vandermonde matrix, named after Alexandre-Théophile Vandermonde, is a matrix with the terms of a geometric progression in each row, i.e., an m × n matrix.</p>
<p>
1&nbsp;&nbsp;a&nbsp;&nbsp;a<sup>2</sup>&nbsp;a<sup>3</sup> . . . . . . . . . . . . . . . .&nbsp;a<sup>(n-1)</sup><br/>
1&nbsp;&nbsp;b&nbsp;&nbsp;b<sup>2</sup>&nbsp;b<sup>3</sup> . . . . . . . . . . . . . . . .&nbsp;b<sup>(n-1)</sup><br/>
1&nbsp;&nbsp;c&nbsp;&nbsp;c<sup>2</sup>&nbsp;c<sup>3</sup> . . . . . . . . . . . . . . . .&nbsp;c<sup>(n-1)</sup><br/>
. . .<br/>
. . .<br/>
. . .<br/>
1&nbsp;&nbsp;m&nbsp;&nbsp;m<sup>2</sup>&nbsp;m<sup>3</sup> . . . . . . . . . . . . . . . .&nbsp;m<sup>(n-1)</sup><br/>
</p>
<p>
Vandermonde matrices are used in linear algebra (Hermite interpolation), DFT (discrete Fourier transforms) and Group theory.</p>
<p>
They are also used in some forms of BCH and Reed–Solomon error correction codes.</p>
<p>
These are an important group of error-correcting codes which have many important applications, which include technologies such as CDs, DVDs, Blu-ray Discs, QR Codes, data transmission technologies such as DSL and WiMAX, broadcast systems such as DVB and ATSC, and storage systems such as RAID 6; they are also used in satellite communication.</p>


The PYTHON (numpy) code is quite straight forward

```python
def vander(x,N):
  x = np.asarray(x)
  if x.ndim != 1:
    raise ValueError("x must be a 1-D array or sequence")
  v = np.empty((len(x), N), dtype=np.promote_types(x.dtype, int))
  if N > 0:
    v[:,0] = 1
  if N > 1:
    v[:, 1:] = x[:, None]
    np.multiply.accumulate(v[:, 1:], out=v[:, 1:], axis=1)
  return v

```

But the accumulate function is very complex, written in C and is 347 lines long.

In [ ]:
x = nr.rand(1000); 

In [ ]:
np.vander(x,50)

The Julia version is much closer to algorithm

```
function vander(x, N::Int)
  x = convert(AbstractVector, x)
  M = length(x)
  v = Array(promote_type(eltype(x),Int), M, N)
  if N > 0 
    v[:, 1] = 1
  end
  if N > 1
    for i = 2:N
      v[:,i] = x
    end
    accumulate(*,v,v)
  end
  return v
end

function accumulate(op, input, output)
  M, N = size(input)
  for i = 2:N
    for j = 1:M
      output[j,i] = op(input[j,i], input[j,i-1])
    end
  end
end

```

The accumulate function is written in Julia and only 6 lines of code

In [ ]:
jl.eval('pwd()')

In [ ]:
jl.call('include("./code/vander.jl")')

In [ ]:
jl.eval("xx = rand(1000);")
jl.eval("@timed vander(xx,50)")

---

# Using the Cosmology module

In [ ]:
jl.eval('using Cosmology')

In [ ]:
# Universe can be Open (hyperbolic), Closed (elliptic) or Flat (parabolic)
#
# Ωk     : Curvature density
# Ωm     : Matter density
# Ωr     : Radiation density = Ωγ + Ων
#
# If Ωr is not specified these can be used to compute it.
# Tcmb   : CMB temperature (K), used to compute Ωγ
# Neff   : Effective number of massless neutrinos, used to compute Ων

jl.eval('csm = cosmology(OmegaK=0.1,OmegaM=0.26,Tcmb=3.1,Neff=3)')


In [ ]:
jl.eval('names(csm)')

In [ ]:
print "Hubble's constant is {}".format(jl.eval('csm.h'))

In [ ]:
print "Age of the universe is {:0.2f} gyr".format(jl.eval('hubble_time_gyr(csm,0)'))

In [ ]:
# Use redshift of 1.3 (=> 30%) in examples below.

rsp = jl.eval('rsj = 1.3')  

In [ ]:
# The angular diameter distance to an object is defined in terms of the object's actual size, x, 
# and the angular size of the object as viewed from earth.
#
# https://ned.ipac.caltech.edu/level5/Hogg/Hogg_contents.html

s = "Angular diameter distance {:0.2f} mpc to an object at a redshift of {}"
print s.format(jl.eval('angular_diameter_dist_mpc(csm, rsj)'), rsp)


In [ ]:
# Lookback time tL to an object is the difference between the age of the Universe now (at observation)
# and the age of the Universe at the time the light reaching us was emitted

s = "Difference between age at redshift {} and the age at redshift {} in {:0.2f} gyr"
print s.format(0, rsp, jl.eval(('lookback_time_gyr(csm, rsj)')))


---

# Asian option price
( Compare with the Python code: _asianOpt.py_ )

In [ ]:
jl.eval("""
function asianOpt(N=1000, T=100; S0=100.0, K=100.0, r=0.05, q=0.0, v=0.2, tma=0.25	) 

# European Asian option.  
# Euler and Milstein discretization for Black-Scholes.

  dt = tma/T;      # Time increment

  S = zeros(Float64,T);
  A = zeros(Float64,N);

# Main calculation loop

  for n = 1:N
    S[1] = S0 
    dW = randn(T)*sqrt(dt);
    for t = 2:T
      z0 = (r - q - 0.5*v*v)*S[t-1]*dt;
      z1 = v*S[t-1]*dW[t];
      z2 = 0.5*v*v*S[t-1]*dW[t]*dW[t];
      S[t] = S[t-1] + z0 + z1 + z2;
    end
    A[n] = mean(S);
  end

# Define the payoff and calculate price

  P = zeros(Float64,N);
  [ P[n] = max(A[n] - K, 0) for n = 1:N ];
  price = exp(-r*tma)*mean(P);

end
""")

In [ ]:
rts = jl.eval('@timed asianOpt(1000000,100; K=102.0)')
rts

In [ ]:
print "Option price is ", rts[0]
print "Time taken was  ", rts[1], "sec."